
### Title: Homework7

### Author: Aishwarya Saibewar

### Date: 11/09/2023

In [2]:
pip install geopandas

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import altair as alt
import geopandas as gpd
import osmnx as ox
import folium
from folium import plugins
from vega_datasets import data
import addfips
af = addfips.AddFIPS()

## Healthcare in Seattle

In [4]:
# get the Seattle geometry
seattle = ox.geocode_to_gdf('R237385', by_osmid = True)

# collect list of healthcare
health = ox.features_from_polygon(seattle["geometry"].iloc[0], tags={"healthcare":True})
health = health[health.geom_type == 'Polygon'] # filter to only polygon geometires
health = gpd.GeoDataFrame.from_features(health) # convert to GeoDataFrame

In [5]:
health.head()

/Users/aishwaryasaibewar/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,geometry,addr:city,addr:housenumber,addr:postcode,addr:street,amenity,emergency,gnis:feature_id,healthcare,healthcare:speciality,...,url,blood:plasma,blood:platelets,blood:stemcells,blood:whole,wifi,wikimedia_commons,owner,ways,type
0,"POLYGON ((-122.33946 47.71588, -122.33555 47.7...",Seattle,None,None,None,hospital,yes,1509819,hospital,None,...,None,None,None,None,None,None,None,None,None,None
1,"POLYGON ((-122.32698 47.69904, -122.32699 47.6...",Seattle,9709,98115,3rd Avenue Northeast,clinic,None,None,clinic,None,...,None,None,None,None,None,None,None,None,None,None
2,"POLYGON ((-122.32093 47.61072, -122.32129 47.6...",Seattle,1401,98104,Madison Street,clinic,None,None,clinic,None,...,None,None,None,None,None,None,None,None,None,None
3,"POLYGON ((-122.30634 47.56474, -122.30767 47.5...",Seattle,1660,98108,South Columbian Way,hospital,yes,None,hospital,None,...,None,None,None,None,None,None,None,None,None,None
4,"POLYGON ((-122.30379 47.65597, -122.30383 47.6...",Seattle,4060,98116,East Stevens Way Northeast,clinic,None,2461226,clinic,None,...,None,None,None,None,None,None,None,None,None,None


In [6]:
health.shape

/Users/aishwaryasaibewar/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(94, 123)

In [7]:
#health['healthcare'].unique()

In [8]:
selected_health_dept = ['hospital', 'clinic', 'pharmacy','dentist']

health_dataset = health[health['healthcare'].isin(selected_health_dept)]
health_dataset.shape

(69, 123)

In [9]:

m = folium.Map(location=[47.60772,-122.34206], 
               tiles='cartodbpositron',
               zoom_start=11, width= 1200, height=600, control_scale=True)
marker_cluster = plugins.MarkerCluster().add_to(m)

for idx, row in health_dataset.iterrows():

    x, y = row['geometry'].exterior.coords.xy
    lat,lon = y[0], x[0]
    data_healthcare = row['healthcare']
    healthcare_name = row['name']
    healthcare_address = row['addr:street']
    
    
    if data_healthcare == 'hospital':
        icon = folium.Icon(icon = 'hospital', color = 'blue', prefix = 'fa')
    elif data_healthcare == 'clinic':
        icon = folium.Icon(icon = 'c', color = 'orange', prefix = 'fa')
    elif data_healthcare == 'pharmacy':
        icon = folium.Icon(icon = 'p', color = 'green', prefix = 'fa')
    elif data_healthcare == 'dentist':
        icon = folium.Icon(icon = 'tooth', color = 'red', prefix = 'fa')    
        
    if data_healthcare == 'hospital':
        healthcare_type = 'Hospital' 
    elif data_healthcare == 'clinic':
        healthcare_type = 'Clinic'
    elif data_healthcare == 'pharmacy':
        healthcare_type = 'Pharmacy'
    elif data_healthcare == 'dentist':
        healthcare_type = 'Dental Care'
        
    # Create a custom tooltip
    tooltip_text = f"{healthcare_name}<br>Street: {healthcare_address}"

        # Add marker to the map
    folium.Marker(location=[lat, lon], 
                  icon=icon,
                  tooltip=tooltip_text,
                  tags=[healthcare_type]
    ).add_to(m)


categories = ['Hospital','Clinic','Pharmacy','Dental Care']
plugins.TagFilterButton(categories).add_to(m)



title_html = '''
              <h2 align="center" style="font-size:20px"><b>Healthcare facilities in Seattle</b></h2>
             ''' 


m.get_root().html.add_child(folium.Element(title_html))

m



In [10]:
m.save('healthcare.html')

In [11]:
with open("healthcare.html", "r") as html_file:
    html_content = html_file.read()
    
description = '''<!DOCTYPE html>
<html>
<head>
    <title>Healthcare facilities in Seattle</title>
    <style>
        body {
            padding-left: 30px; 
            padding-right: 30px;
        }
    </style>
</head>
<body>
    <h1><center>Healthcare facilities in Seattle</center></h1>
    <p style="font-size: 15px;">This map helps in locating the healthcare facilities available in Seattle. This map uses the healthcare data from the “Open Street Map”. The healthcare facilities in Seattle are superimposed on top of the geographical base map of the city, and their exact locations are indicated with markers. The locations of healthcare facilities on the map represent hospitals, clinics, pharmacies, and dental care. The marker icons and distinctive colors reflect the four distinct healthcare categories, making it easier for viewers to differentiate between various healthcare facilities. Additionally, a filter has been incorporated in the map, to facilitate the users in selecting a specific healthcare category and identifying their corresponding location. A tooltip is added to indicate the healthcare facility's name, alongside the street address.  The map can also be zoomed in to pinpoint the precise location of the healthcare facility. It is evident from the map that pharmacy and dental care are more prevalent in the North of Seattle, while hospitals and clinics are concentrated in the city's center. </p>
</body>
<div class="content">
<!-- Page content -->
</div>
</html>'''

html_content = html_content.replace('<body>', f'<body>\n{description}')

with open("healthcare.html", "w") as html_file:
    html_file.write(html_content)